In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os

In [ ]:
def datestr2sec(date):
    secs = [3600,60,1]
    return sum([secs[i]*float(el) for i,el in enumerate(date.split(':'))])

def sec2datestr(seconds):
    secs = [3600, 60, 1]
    hours = int(seconds // secs[0])
    seconds = seconds % secs[0]
    mins = int(seconds // secs[1])
    seconds = seconds % secs[1]
    return str(hours)+':'+str(mins)+':'+str(int(seconds))

print(datestr2sec(sec2datestr(12)))
print(datestr2sec('00:01:23'))

In [ ]:
directory_bim = "/home/helge/Programs/matsim/matsim_results/ConvCrit/115997reqs/bim_10000trcap"
N_trips = 115997

def getTripTimes(directory):
    gammas = []
    trips = []
    drt_wait_times = []
    
    for root, subdirs, files in os.walk(directory):
        for file in files:
            if file == 'output_trips.csv.gz':
                request = re.match(".*gamma(\d*_\d*)", root)
                gammas.append(float(request.group(1).replace('_', '.')))
                df = pd.read_csv(os.path.join(root, file), sep=';', compression='gzip').loc[:,['person','trav_time','wait_time','modes']]
                df['trav_time'] = df['trav_time'].apply(datestr2sec)
                df['wait_time'] = df['wait_time'].apply(datestr2sec)
                df.set_index('person', inplace=True)
#                 display(df.sort_values('euclidean_distance'))
#                 display(df.groupby('modes').count())
                trips.append(df)
            if file == '0.drt_trips_drt.csv':
                df = pd.read_csv(os.path.join(root, file), sep=';').loc[:,['personId','waitTime']]
                drt_wait_times.append(df.groupby('personId').sum())

    return np.array(gammas), trips, drt_wait_times
 

gammas, trip_dfs, drt_wait_times = getTripTimes(directory_bim)

for i,g in enumerate(gammas):
    print('Gamma: ', g)
    trip_dfs[i]['pure_trav_time'] = trip_dfs[i]['trav_time'] - trip_dfs[i]['wait_time']
    trip_dfs[i]['pure_trav_time'] = trip_dfs[i]['pure_trav_time'] + drt_wait_times[i]['waitTime']
#     trip_dfs[i] = trip_dfs[i][trip_dfs[i]['pure_trav_time'] < 3600]
    mean_time = trip_dfs[i]['pure_trav_time'].mean()
    print('Mean travel time: ', mean_time, ', formatted: ', sec2datestr(mean_time))
    print('ratio of successfull trips: ', len(trip_dfs[i])/N_trips)
    print('Average wait time: ', drt_wait_times[i]['waitTime'].mean())
    mode_count = trip_dfs[i].groupby('modes').count()['trav_time']
    print('Pt in trip ratio: ', sum([count for key,count in mode_count.to_dict().items() if 'pt' in key])/mode_count.sum())
#     display(mode_count)
    plt.figure()
    trip_dfs[i]['pure_trav_time'].plot.hist(bins=24, range=(0, 3600))
    plt.xlabel('seconds')
    plt.title('Gamma: {}'.format(g))
    print('-------\n')

plt.show()

In [ ]:
directory_drt = "/home/helge/Programs/matsim/matsim_results/ConvCrit/115997reqs/drt/gammaInfty"

drt_df = pd.read_csv(os.path.join(directory_drt, 'output_trips.csv.gz'), sep=';', compression='gzip').loc[:,['person','trav_time','wait_time']]
drt_df['pure_trav_time'] = drt_df['trav_time'].apply(datestr2sec)# - drt_df['wait_time'].apply(datestr2sec)
drt_df.set_index('person', inplace=True)

# drt_df2 = pd.read_csv('/home/helge/Programs/matsim/matsim_results/ConvCrit/115997reqs/drt/gammaInfty/ITERS/it.0/0.drt_trips_drt.csv', sep=';').loc[:,['personId', 'waitTime', 'travelTime']]
# drt_df2 = drt_df2.groupby('personId').sum()
# drt_df['pure_trav_time'] = drt_df['pure_trav_time'] + drt_df2['waitTime']

print('ratio of successfull trips: ', len(drt_df)/N_trips)
print('Mean travel time: ', drt_df['pure_trav_time'].mean())
print('Mean wait time: ', drt_df['wait_time'].apply(datestr2sec).mean())

drt_df['pure_trav_time'].plot.hist(bins=24, range=(0, 3600))
plt.title(r'Gamma: $\infty$')
plt.show()

In [ ]:
mode_count = trip_dfs[0].groupby('modes').count()['trav_time']
print(mode_count)
print(sum([count for key,count in mode_count.to_dict().items() if 'drt' in key]))